In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.kernel_approximation import RBFSampler
from sklearn.linear_model import RidgeClassifier

In [2]:
# Importing Important Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [36]:
df_train = pd.read_csv('HP_with_lag.csv', parse_dates = ["Date"])
df_train.head()

,Date,Rain,T_max,T_min,Tourism,Travel_index,Travel,Bus & Rail,Tourist Destinations,Travel Agencies,value_lag_1,value_lag_2,value_lag_3,value_lag_4,value_lag_5,Tourist Arrivals
0,2010-06-01,1.290605,1.447768,0.783780,5.072035,-1.488372,2.805842,5.639656,2.870036,0,1592719.0,1903137.0,1428569.0,869384.0,757377.0,1808553.0
1,2010-07-01,2.680181,0.759656,1.136959,2.085816,-1.488372,0.427514,1.340389,0.645100,4,1808553.0,1592719.0,1903137.0,1428569.0,869384.0,1088438.0
2,2010-08-01,1.969188,0.411474,1.395857,0.979809,-1.488372,-0.067970,0.363283,0.131653,0,1088438.0,1808553.0,1592719.0,1903137.0,1428569.0,1087424.0
3,2010-09-01,1.306172,0.271211,0.659569,1.422212,3.657025,0.526611,0.754125,-0.210645,0,1087424.0,1088438.0,1808553.0,1592719.0,1903137.0,1196179.0
4,2010-10-01,-0.809294,0.111147,-0.171000,1.201010,3.335437,0.130224,0.558704,1.158547,0,1196179.0,1087424.0,1088438.0,1808553.0,1592719.0,1484130.0


In [37]:
df_test = pd.read_csv('shimla_with_lag.csv', parse_dates = ["Date"])
df_test.head()

,Date,Rain,T_max,T_min,Tourism,Travel_index,Travel,Bus & Rail,Tourist Destinations,Travel Agencies,value_lag_1,value_lag_2,value_lag_3,value_lag_4,value_lag_5,Tourist Arrivals
0,2010-06-01,-0.397005,1.006507,1.006507,4.023852,3.992883,2.372893,1.821047,3.183420,0,390553.0,395110.0,198760.0,174609.0,183556.0,432559.0
1,2010-07-01,-0.275566,0.937469,0.937469,1.835582,1.370189,0.642150,0.787077,1.083396,0,432559.0,390553.0,395110.0,198760.0,174609.0,197096.0
2,2010-08-01,0.765266,0.943605,0.943605,1.085318,0.878434,0.019083,-0.053024,0.411389,2,197096.0,432559.0,390553.0,395110.0,198760.0,222870.0
3,2010-09-01,0.373287,1.081681,1.081681,1.022796,1.042353,0.434461,0.334715,0.831394,4,222870.0,197096.0,432559.0,390553.0,395110.0,207944.0
4,2010-10-01,-0.397005,0.379028,0.379028,1.022796,2.681536,0.365231,0.528584,0.495390,0,207944.0,222870.0,197096.0,432559.0,390553.0,154716.0


In [ ]:
# df["Recreation"] = df["Recreation"].astype(int)
# df["Car & Taxi Rental"] = df["Car & Taxi Rental"].astype(int)

In [5]:
def create_features(df, label=None):
    """
    Creates time series features from datetime index
    """
    df['date'] = df.index
    df['dayofweek'] = df['Date'].dt.dayofweek
    df['quarter'] = df['Date'].dt.quarter
    df['month'] = df['Date'].dt.month
    df['year'] = df['Date'].dt.year
    df['dayofyear'] = df['Date'].dt.dayofyear
    df['dayofmonth'] = df['Date'].dt.day
    df['weekofyear'] = df['Date'].dt.weekofyear

    X = df[['quarter','month','year','Rain', 'T_max', 'T_min', 'Tourism', 'Travel_index', 'Travel', 'Bus & Rail', 'Tourist Destinations', 'value_lag_1', 'value_lag_2', 'value_lag_3', 'value_lag_4', 'value_lag_5', 'Travel Agencies']]
    if label:
        y = df[label]
        return X, y
    return X

In [38]:
df_train = df_train.replace(to_replace="<1",
           value=0.5)

In [39]:
df_test = df_test.replace(to_replace="<1",
           value=0.5)

In [ ]:
df_test.columns.to_list()

['Date',
 'Rain',
 'T_max',
 'T_min',
 'Tourism',
 'Travel_index',
 'Travel',
 'Bus & Rail',
 'Tourist Destinations',
 'Travel Agencies',
 'value_lag_1',
 'value_lag_2',
 'value_lag_3',
 'value_lag_4',
 'value_lag_5',
 'Tourist Arrivals',
 'date',
 'dayofweek',
 'quarter',
 'month',
 'year',
 'dayofyear',
 'dayofmonth',
 'weekofyear']

In [ ]:
df_train.rename(columns = {'Kerala_Tourism':'Tourism'}, inplace = True)
df_train.rename(columns = {'Kerala_Travel':'Travel_index'}, inplace = True)

In [ ]:
split_date = '2020-01-05'
df_train = df.loc[df["Date"] <= split_date].copy()
df_test = df.loc[df["Date"] > split_date].copy()

In [40]:
X_train, y_train = create_features(df_train, label='Tourist Arrivals')

# X, y = create_features(df, label='Tourist Arrivals')

<ipython-input-5-5b9b2c6373f3>:12: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  df['weekofyear'] = df['Date'].dt.weekofyear


In [ ]:
X_train = X_train.drop(["Cluster_ID"], axis = 1)
X_train

KeyError: ignored

In [41]:
X_test, y_test = create_features(df_test, label='Tourist Arrivals')

<ipython-input-5-5b9b2c6373f3>:12: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  df['weekofyear'] = df['Date'].dt.weekofyear


In [10]:
def calculate_rmse(actual_values, predicted_values):
    """
    Calculate Root Mean Square Error (RMSE) between actual and predicted values.

    Parameters:
    - actual_values: Array or list of actual values.
    - predicted_values: Array or list of predicted values.

    Returns:
    - rmse: Root Mean Square Error.
    """
    mse = mean_squared_error(actual_values, predicted_values)
    rmse = np.sqrt(mse)
    return rmse

In [11]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, accuracy_score

In [42]:
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error as mse
from math import sqrt
# X = X
# y = y

# # X['SKU id'] = X['SKU id'].replace({'SKU-': ''}, regex=True)
# # X['Warehouse id'] = X['Warehouse id'].replace({'Wh-': ''}, regex=True)
# #X.head()

# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42, shuffle=False)
# Choose a kernel and set its parameters
gamma = 0.1
n_components = 100
rbf_feature = RBFSampler(gamma=gamma, n_components=n_components)

# Apply the kernel to the training data
x_train_transformed = rbf_feature.fit_transform(X_train)

# Train the Ridge Regression model using the transformed data
ridge_regressor = Ridge()
ridge_regressor.fit(x_train_transformed, y_train)

# Apply the same kernel to the test data
x_test_transformed = rbf_feature.transform(X_test)

# Make predictions on the test data
predictions = ridge_regressor.predict(x_test_transformed)

# Evaluate the regression performance
mse = mean_squared_error(y_test, predictions)
print(f"Mean Squared Error: {mse}")
mape = mean_absolute_percentage_error(y_test, predictions)

print(r2_score(y_test, predictions))
mape

Mean Squared Error: 1002452783089.8676
-125.62744712354569


5.41389043656167

In [43]:
import numpy as np
class ELM:
    def __init__(self, input_size, hidden_size, output_size, activation_function=None):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size

        # Randomly initialize input weights and biases
        self.input_weights = np.random.rand(hidden_size, input_size)
        self.biases = np.random.rand(hidden_size, 1)
        self.output_weights = None  # Will be set during training

        # Set the activation function
        if activation_function is None:
            self.activation_function = self.sigmoid
        else:
            self.activation_function = activation_function
        print("Activation function type:", type(self.activation_function))

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def fit(self, X, y):
        # Compute the hidden layer output
        hidden_output = self.activation_function(np.dot(self.input_weights, X.T) + self.biases)

        # Calculate output weights using the pseudoinverse
        self.output_weights = np.dot(np.linalg.pinv(hidden_output.T), y)

    def predict(self, X):
        # Compute hidden layer output for prediction
        hidden_output = self.activation_function(np.dot(self.input_weights, X.T) + self.biases)

        # Make predictions using output weights
        predictions = np.dot(hidden_output.T, self.output_weights)

        return predictions

    # The following methods are needed for compatibility with scikit-learn

    def get_params(self, deep=True):
        return {'input_size': self.input_size, 'hidden_size': self.hidden_size, 'output_size': self.output_size}

    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            setattr(self, parameter, value)
        return self

# Example usage:
input_size = 17
hidden_size = 20
output_size = 10

# Create ELM model
elm_model = ELM(input_size, hidden_size, output_size)

# Generate random data for demonstration
# X_train = np.random.rand(100, input_size)
# y_train = np.random.rand(100, output_size)

# Train the ELM model
elm_model.fit(X_train, y_train)

# Make predictions on new data
# X_test = np.random.rand(10, input_size)
predictions = elm_model.predict(X_test)

Activation function type: <class 'method'>


In [44]:
mape = mean_absolute_percentage_error(y_test, predictions)
print(calculate_rmse(y_test, predictions))
print(r2_score(y_test, predictions))
mape

965391.1611304457
-116.72550494398993


5.238399033682926

In [45]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error

In [46]:
elm_model = ELM(input_size, hidden_size, output_size)

# Define the parameter grid
param_grid = {
    'hidden_size': [10, 20, 30],
    'activation_function': ['sigmoid', 'tanh'],
    'regularization_strength': [0.1, 0.01, 0.001]
}

# Define your scoring metric (e.g., mean squared error for regression)
scoring = 'neg_mean_squared_error'

# Create the GridSearchCV object
grid_search = GridSearchCV(elm_model, param_grid, cv=5, scoring=scoring)

# Assuming you have a dataset X_train, y_train
grid_search.fit(X_train, y_train)

# Print the best parameters
print("Best parameters found: ", grid_search.best_params_)

# Get the best model
best_elm_model = grid_search.best_estimator_

# Make predictions on new data
predictions = best_elm_model.predict(X_test)

Activation function type: <class 'method'>
Activation function type: <class 'method'>
Activation function type: <class 'method'>
Activation function type: <class 'method'>
Activation function type: <class 'method'>
Activation function type: <class 'method'>
Activation function type: <class 'method'>
Activation function type: <class 'method'>
Activation function type: <class 'method'>
Activation function type: <class 'method'>
Activation function type: <class 'method'>
Activation function type: <class 'method'>
Activation function type: <class 'method'>
Activation function type: <class 'method'>
Activation function type: <class 'method'>
Activation function type: <class 'method'>
Activation function type: <class 'method'>
Activation function type: <class 'method'>
Activation function type: <class 'method'>
Activation function type: <class 'method'>
Activation function type: <class 'method'>
Activation function type: <class 'method'>
Activation function type: <class 'method'>
Activation 

ValueError: ignored

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error

# Create ELM model
elm_model = ELM(input_size, hidden_size, output_size)

# Define the parameter grid
param_grid = {
    'hidden_size': [10, 20, 30],
    'activation_function': ['sigmoid', 'tanh'],
}

# Define your scoring metric (e.g., mean squared error for regression)
scoring = 'neg_mean_squared_error'

# Create the GridSearchCV object
grid_search = GridSearchCV(elm_model, param_grid, cv=5, scoring=scoring)

# Assuming you have a dataset X_train, y_train
grid_search.fit(x_train_transformed, y_train)

# Print the best parameters
print("Best parameters found: ", grid_search.best_params_)

# Get the best model
best_elm_model = grid_search.best_estimator_

# Make predictions on new data
predictions = best_elm_model.predict(x_test_transformed)

Activation function type: <class 'method'>
Activation function type: <class 'method'>
Activation function type: <class 'method'>
Activation function type: <class 'method'>
Activation function type: <class 'method'>
Activation function type: <class 'method'>
Activation function type: <class 'method'>
Activation function type: <class 'method'>
Activation function type: <class 'method'>
Activation function type: <class 'method'>
Activation function type: <class 'method'>
Activation function type: <class 'method'>
Activation function type: <class 'method'>
Activation function type: <class 'method'>
Activation function type: <class 'method'>
Activation function type: <class 'method'>
Activation function type: <class 'method'>
Activation function type: <class 'method'>
Activation function type: <class 'method'>
Activation function type: <class 'method'>
Activation function type: <class 'method'>
Activation function type: <class 'method'>
Activation function type: <class 'method'>
Activation 

ValueError: ignored

In [47]:
from sklearn.neural_network import MLPRegressor
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np

# Generate synthetic regression data for demonstration
# X, y = make_regression(n_samples=1000, n_features=10, noise=0.1, random_state=42)

# # Split the data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the ELM model using MLPRegressor
elm_model = MLPRegressor(hidden_layer_sizes=(100,), activation='identity', solver='adam', random_state=42)

# Train the ELM model
elm_model.fit(X_train, y_train)

# Make predictions on the test set
predictions = elm_model.predict(X_test)

# Evaluate the performance using Mean Squared Error
mse = mean_squared_error(y_test, predictions)
print(f"Mean Squared Error: {mse}")


Mean Squared Error: 9745421603.739891


In [48]:
mape = mean_absolute_percentage_error(y_test, predictions)
print(calculate_rmse(y_test, predictions))
print(r2_score(y_test, predictions))
mape

98718.90195772992
-0.23101843761713048


0.4018390673828191

In [49]:
param_grid = {
    'hidden_layer_sizes': [(50,), (100,), (200,)],  # You can add more values
    'alpha': [0.0001, 0.001, 0.01]  # Regularization strength
}

# Create the ELM model (MLPRegressor)
elm_model = MLPRegressor(activation='identity', solver='adam', random_state=42)

# Create the GridSearchCV object
grid_search = GridSearchCV(elm_model, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

# Perform hyperparameter tuning on the training data
grid_search.fit(X_train, y_train)

# Print the best parameters and their corresponding MSE
print("Best parameters found: ", grid_search.best_params_)
print("Best MSE on validation data: ", -grid_search.best_score_)

# Get the best model
best_elm_model = grid_search.best_estimator_

# Evaluate the best model on the test set
test_predictions = best_elm_model.predict(X_test)
test_mse = mean_squared_error(y_test, test_predictions)
print("Mean Squared Error on test data: ", test_mse)

Best parameters found:  {'alpha': 0.01, 'hidden_layer_sizes': (50,)}
Best MSE on validation data:  161935526763.9607
Mean Squared Error on test data:  8841214779.830725


In [50]:
mape = mean_absolute_percentage_error(y_test, test_predictions)
print(calculate_rmse(y_test, predictions))
print(r2_score(y_test, predictions))
mape

98718.90195772992
-0.23101843761713048


0.38229831664424546

In [ ]:
from sklearn.preprocessing import StandardScaler

# Scale the features
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train_transformed)
x_test_scaled = scaler.transform(x_test_transformed)

In [28]:
from sklearn.pipeline import Pipeline
# Define the joint parameter space for both kernel and ELM
param_grid = {
    'kernel__gamma': [0.1, 1, 10],  # Adjust gamma for the RBF kernel
    'elm__hidden_layer_sizes': [(50,), (100,), (200,)],  # Hidden layer sizes
    'elm__alpha': [0.0001, 0.001, 0.01],  # Regularization strength for ELM
}

# Create the RBF kernel approximation
rbf_feature = RBFSampler(random_state=42)

# Create the ELM model (MLPRegressor) with RBF kernel
elm_model = MLPRegressor(activation='identity', solver='adam', random_state=42)

# Create a pipeline with the kernel and ELM
pipeline = Pipeline([
    ('kernel', rbf_feature),
    ('elm', elm_model)
])

# Create the GridSearchCV object
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

# Perform hyperparameter tuning on the training data
grid_search.fit(X_train, y_train)

# Print the best parameters and their corresponding MSE
print("Best parameters found: ", grid_search.best_params_)
print("Best MSE on validation data: ", -grid_search.best_score_)

# Get the best model
best_model = grid_search.best_estimator_

# Evaluate the best model on the test set
test_predictions = best_model.predict(X_test)
test_mse = mean_squared_error(y_test, test_predictions)
print("Mean Squared Error on test data: ", test_mse)

Best parameters found:  {'elm__alpha': 0.01, 'elm__hidden_layer_sizes': (200,), 'kernel__gamma': 1}
Best MSE on validation data:  920997436069.2073
Mean Squared Error on test data:  1551038888908.9885


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [29]:
mape = mean_absolute_percentage_error(y_test, test_predictions)
print(calculate_rmse(y_test, predictions))
print(r2_score(y_test, predictions))
mape

981207.5632342098
-5.499735312490523


0.9999827856995986

In [ ]:
# Define the Elastic Net model
elastic_net_model = ElasticNet()

# Define the hyperparameters to search over
param_grid = {
    'alpha': [0.1, 1.0, 10.0],
    'l1_ratio': [0.1, 0.5, 0.9]
}

# Use grid search with cross-validation
grid_search = GridSearchCV(elastic_net_model, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(X_train, y_train)

# Print the best hyperparameters
best_params = grid_search.best_params_
print(f"Best Hyperparameters: {best_params}")

# Apply the best model to the test data
best_model = grid_search.best_estimator_
predictions = best_model.predict(X_test)

# Evaluate the regression performance
mse_best_model = mean_squared_error(y_test, predictions)
print(f"MSE (Best Model): {mse_best_model}")
r2 = r2_score(y_test, predictions)
print(f"R2 Score (EN): {r2}")
mape = mean_absolute_percentage_error(y_test, predictions)
print(r2_score(y_test, predictions))
mape

In [ ]:
mape = mean_absolute_percentage_error(y_test, test_predictions)
print(calculate_rmse(y_test, predictions))
print(r2_score(y_test, predictions))
mape